In [ ]:
# 건물별 고유혼잡도

import io
import pandas as pd
import numpy as np
import datetime as dt
from pandas import Series, DataFrame


# 학부 개설과목 및 수강인원
from google.colab import drive
drive.mount('/content/drive')
filename_org = '/content/drive/MyDrive/데이터톤/보고서 작성 시 사용할 최종 파일 /데이터 및 전처리/강의별 수강인원 전처리 최종.txt'
org = pd.read_csv(filename_org, sep = "\t",error_bad_lines=False) 
df = org.drop(columns=["과목명", "room"])
df = df.dropna(axis=0)
df0 = df[df['day'] != '토']


# 요일 인덱스
df1 = df0.replace(['월','화','수','목','금'],[0,1,2,3,4]) 


# 건물 인덱스
df2 = df1.replace(['CJ법학관','L-P','RnD센터','공학관','과도관','교양관','교육관','국제관','로봇융합관','미디어관',
                   '미래융합','법학관구관','법학관신관','법학도서관','사대본관','사대신관','생명과학관(동관)','생명과학관(서관)',
                   '서관','신공학관','아산이학관','우정간호학관','의과대학본관','의대','이학관별관','정경관','정보통신관','중앙도서관(대학원)',
                   '창의관','하나과학관','하나스퀘어','현차관'],[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31])


# 요일 결정
range = pd.date_range("09:00", "22:00", freq="30min").strftime('%H:%M:%S')
df3 = df2[df2['day'] == 4] # 요일 인덱스 입력
df3 = df3.reset_index(drop=True)
df3['start'] = pd.to_datetime(df3['start'], format="%H:%M")
df3['end'] = pd.to_datetime(df3['end'], format="%H:%M")
dfFin = pd.DataFrame(data = list(range), index = None, columns = ['time'])
dfFin['time'] = pd.to_datetime(dfFin['time'], format = '%H:%M:%S')


# 혼잡도 산정
num = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31]
for k in num: 
  df4 = df3[df3['buliding'] == k]
  congestion = [] # 해당 요일의 전체 시간대에 대한 혼잡도 리스트
  for i in dfFin.index:
    sum = 0 # 초기화
    for j in df4.index:
      if dfFin["time"][i] >= df4["start"][j] and dfFin["time"][i] <= df4["end"][j]:
        sum += df4["student"][j]
    congestion.append(sum)
  df5 = pd.DataFrame(congestion,columns=[k])
  dfFin = pd.concat([dfFin,df5], axis = 1) 


# 해당 요일의 고유 혼잡도
dfFin.rename(columns = {0: "CJ법학관",1: "L-P",2: "R&D센터",3: "공학관",4: "과학도서관",5: "우당교양관",6: "운초우선교육관",7: "국제관",8: "로봇융합관",9: "미디어관",10: "미래융합기술관",
                        11: "법학관(구관)",12: "법학관(신관)",13: "해송법학도서관",14: "사범대학본관",15: "사범대학신관",16: "생명과학관(동관)",17: "생명과학관(서관)",18: "문과대학(서관)",19: "신공학관",
                        20: "아산이학관",21: "우정간호학관",22: "의과대학본관",23: "의과대학문숙의학관",24: "이학관별관",25: "정경관",26: "우정정보통신관",27: "중앙도서관(대학원)",28: "창의관",
                        29: "하나과학관",30: "하나스퀘어",31: "현대자동차경영관"}, inplace = True)


# 강의실 없는 건물 추가한 최종 고유 혼잡도
nonClass = ["4.18기념관","CJ식품안전관","SK미래관","동원글로벌리더십홀","메디힐지구환경관","백주년기념삼성관","애기능생활관","의과대학제1의학관","인촌기념관","제2실험관",
            "중앙광장","중앙도서관(신관)","타이거플라자","환경실험관"] # 강의실 없는 건물 리스트


dummy = np.zeros(shape=(27,), dtype=np.int8) 
for bdName in nonClass:
  dfNonclass = pd.DataFrame(data = dummy, columns=[bdName])
  dfFin = pd.concat([dfFin,dfNonclass], axis = 1)
print(dfFin)


# CSV 저장
dfFin.to_csv('금요일 고유혼잡도(인원수).csv')




In [ ]:
# 건물별 주변혼잡도

import io
import pandas
import requests
import json
import pandas as pd
import numpy as np
from haversine import haversine



# 건물 별 위도경도 데이터
from google.colab import drive
drive.mount('/content/drive')
filename_bd = '/content/drive/MyDrive/데이터톤/data/강의실_위도경도.xlsx'
bd = pd.read_excel(filename_bd)
bd.rename(columns = {'lon,lat' : 'location'}, inplace = True)
bd['lon'] = bd.location.str.split(',').str[0]
bd['lat'] = bd.location.str.split(',').str[1]



# 요일 별 고유혼잡도 데이터
rawMon = pandas.read_csv('/content/drive/MyDrive/데이터톤/전처리/혼잡도 과정데이터(사용X)/월요일 고유혼잡도(인원수).csv')
rawTue = pandas.read_csv('/content/drive/MyDrive/데이터톤/전처리/혼잡도 과정데이터(사용X)/화요일 고유혼잡도(인원수).csv')
rawWed = pandas.read_csv('/content/drive/MyDrive/데이터톤/전처리/혼잡도 과정데이터(사용X)/수요일 고유혼잡도(인원수).csv')
rawThu = pandas.read_csv('/content/drive/MyDrive/데이터톤/전처리/혼잡도 과정데이터(사용X)/목요일 고유혼잡도(인원수).csv')
rawFri = pandas.read_csv('/content/drive/MyDrive/데이터톤/전처리/혼잡도 과정데이터(사용X)/금요일 고유혼잡도(인원수).csv')


# 최대-최소 정규화
def minmax_norm(df):
  return (df - df.min()) / ( df.max() - df.min())


# 직선거리 200미터 이하 건물 추출
def distance_under200(building):
    building_lon_lat = pandas.read_excel('/content/drive/My Drive/데이터톤/data/강의실_위도경도.xlsx')

    current_lon = building['lon']
    current_lat = building['lat']

    df_final = pandas.DataFrame(columns=['building', 'distance'])

    for idx, row in building_lon_lat.iterrows():
        start = (float(current_lat), float(current_lon))
        goal = (float(row['lon,lat'].split(',')[1]),float(row['lon,lat'].split(',')[0]))

        if haversine(start, goal) < 0.2:
            request_str = 'http://router.project-osrm.org/route/v1/foot/'+str(current_lat)+','+str(current_lat)+';'+row['lon,lat']+'?overview=false'
            
            r = requests.get(request_str)
            routes = json.loads(r.content)
            distance = routes.get("routes")[0]['distance']
            df_final.loc[len(df_final)] = [row['building'],distance]

    df_final.sort_values('distance', inplace=True)
    df_final['distance'] = minmax_norm(df_final['distance'])

    #print ("distance_under200")
    #print (df_final)
    return df_final



# 기준 건물과의 거리에 비례하여 각 (기준)건물별 고유혼잡도에 주변혼잡도 합산
range = pd.date_range("09:00", "22:00", freq="30min").strftime('%H:%M:%S')
dfWeight = pd.DataFrame(data = list(range), index = None, columns = ['time'])
dfWeight['time'] = pd.to_datetime(dfWeight['time'], format = '%H:%M:%S') # 최종 데이터프레임

for i in bd.index:
  building = bd.loc[i]
  dfTemp = distance_under200(building)
  dfTemp = dfTemp.reset_index(drop=True)
  conAround = []
  result = 0
  
  for j in dfWeight.index:
  
    idx = 0
    for name in dfTemp['building']:
      result += (rawMon.loc[j, name]) * (1+(dfTemp.loc[idx, 'distance'])) # 요일 설정(rawXXX)
      idx += 1
    conAround.append(result)
  
  dfAround = pd.DataFrame(conAround,columns=[i])
  dfWeight = pd.concat([dfWeight,dfAround], axis = 1)


# 인덱스 건물명으로 변경
dfWeight.rename(columns = {0: "CJ법학관",1: "L-P",2: "R&D센터",3: "공학관",4: "과학도서관",5: "우당교양관",6: "운초우선교육관",7: "국제관",8: "로봇융합관",9: "미디어관",10: "미래융합기술관",
                        11: "법학관(구관)",12: "법학관(신관)",13: "해송법학도서관",14: "사범대학본관",15: "사범대학신관",16: "생명과학관(동관)",17: "생명과학관(서관)",18: "문과대학(서관)",19: "신공학관",
                        20: "아산이학관",21: "우정간호학관",22: "의과대학본관",23: "의과대학문숙의학관",24: "이학관별관",25: "정경관",26: "우정정보통신관",27: "중앙도서관(대학원)",28: "창의관",
                        29: "하나과학관",30: "하나스퀘어",31: "현대자동차경영관",32: "4.18기념관",33: "CJ식품안전관", 34: "SK미래관", 35: "동원글로벌리더십홀", 36: "메디힐지구환경관",
                        37: "백주년기념삼성관", 38: "애기능생활관", 39: "의과대학제1의학관", 40: "인촌기념관", 41: "제2실험관", 42: "중앙광장", 43: "중앙도서관(신관)", 44: "타이거플라자",
                        45: "환경실험관"}, inplace = True)

dfWeight.head()

# CSV 저장
dfWeight.to_csv('월요일 주변혼잡도(인원수).csv')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 건물별 1차혼잡도

import io
import pandas as pd
import numpy as np


# 요일 별 고유혼잡도 데이터
from google.colab import drive
drive.mount('/content/drive')
uniMon = pandas.read_csv('/content/drive/MyDrive/데이터톤/전처리/혼잡도 과정데이터(사용X)/월요일 고유혼잡도(인원수).csv')
uniTue = pandas.read_csv('/content/drive/MyDrive/데이터톤/전처리/혼잡도 과정데이터(사용X)/화요일 고유혼잡도(인원수).csv')
uniWed = pandas.read_csv('/content/drive/MyDrive/데이터톤/전처리/혼잡도 과정데이터(사용X)/수요일 고유혼잡도(인원수).csv')
uniThu = pandas.read_csv('/content/drive/MyDrive/데이터톤/전처리/혼잡도 과정데이터(사용X)/목요일 고유혼잡도(인원수).csv')
uniFri = pandas.read_csv('/content/drive/MyDrive/데이터톤/전처리/혼잡도 과정데이터(사용X)/금요일 고유혼잡도(인원수).csv')


# 요일 별 주변혼잡도 데이터
ardMon = pandas.read_csv('/content/drive/MyDrive/데이터톤/전처리/혼잡도 과정데이터(사용X)/월요일 주변혼잡도(인원수).csv')
ardTue = pandas.read_csv('/content/drive/MyDrive/데이터톤/전처리/혼잡도 과정데이터(사용X)/화요일 주변혼잡도(인원수).csv')
ardWed = pandas.read_csv('/content/drive/MyDrive/데이터톤/전처리/혼잡도 과정데이터(사용X)/수요일 주변혼잡도(인원수).csv')
ardThu = pandas.read_csv('/content/drive/MyDrive/데이터톤/전처리/혼잡도 과정데이터(사용X)/목요일 주변혼잡도(인원수).csv')
ardFri = pandas.read_csv('/content/drive/MyDrive/데이터톤/전처리/혼잡도 과정데이터(사용X)/금요일 주변혼잡도(인원수).csv')


# 강의실 유무에 따른 건물 구분
rangeO = ['CJ법학관','L-P','R&D센터','공학관','과학도서관','우당교양관','운초우선교육관','국제관','로봇융합관','미디어관',
          '미래융합기술관','법학관(구관)','법학관(신관)','해송법학도서관','사범대학본관','사범대학신관','생명과학관(동관)','생명과학관(서관)',
           '문과대학(서관)','신공학관','아산이학관','우정간호학관','의과대학본관','의과대학문숙의학관','이학관별관','정경관','우정정보통신관','중앙도서관(대학원)', 
           '창의관','하나과학관','하나스퀘어','현대자동차경영관']
rangeX = ["4.18기념관","CJ식품안전관","SK미래관","동원글로벌리더십홀","메디힐지구환경관","백주년기념삼성관","애기능생활관","의과대학제1의학관","인촌기념관","제2실험관",
            "중앙광장","중앙도서관(신관)","타이거플라자","환경실험관"]


# 월요일
oClassMonUni = uniMon[rangeO]
xClassMonUni = uniMon[rangeX]
oClassMonArd = ardMon[rangeO]
xClassMonArd = ardMon[rangeX]

oClassMon = oClassMonUni + oClassMonArd
xClassMon = xClassMonArd * 2

monFin = pd.concat([oClassMon,xClassMon], axis = 1) # 월요일 1차혼잡도(인원수)


# 화요일
oClassTueUni = uniTue[rangeO]
xClassTueUni = uniTue[rangeX]
oClassTueArd = ardTue[rangeO]
xClassTueArd = ardTue[rangeX]

oClassTue = oClassTueUni + oClassTueArd
xClassTue = xClassTueArd * 2

tueFin = pd.concat([oClassTue,xClassTue], axis = 1) # 화요일 1차혼잡도(인원수)


# 수요일
oClassWedUni = uniWed[rangeO]
xClassWedUni = uniWed[rangeX]
oClassWedArd = ardWed[rangeO]
xClassWedArd = ardWed[rangeX]

oClassWed = oClassWedUni + oClassWedArd
xClassWed = xClassWedArd * 2

wedFin = pd.concat([oClassWed,xClassWed], axis = 1) # 수요일 1차혼잡도(인원수)


# 목요일
oClassThuUni = uniThu[rangeO]
xClassThuUni = uniThu[rangeX]
oClassThuArd = ardThu[rangeO]
xClassThuArd = ardThu[rangeX]

oClassThu = oClassThuUni + oClassThuArd
xClassThu = xClassThuArd * 2

thuFin = pd.concat([oClassThu,xClassThu], axis = 1) # 목요일 1차혼잡도(인원수)


# 금요일
oClassFriUni = uniFri[rangeO]
xClassFriUni = uniFri[rangeX]
oClassFriArd = ardFri[rangeO]
xClassFriArd = ardFri[rangeX]

oClassFri = oClassFriUni + oClassFriArd
xClassFri = xClassFriArd * 2

friFin = pd.concat([oClassFri,xClassFri], axis = 1) # 금요일 1차혼잡도(인원수)


# CSV 저장
monFin.to_csv('월요일 최종혼잡도(인원수).csv')
tueFin.to_csv('화요일 최종혼잡도(인원수).csv')
wedFin.to_csv('수요일 최종혼잡도(인원수).csv')
thuFin.to_csv('목요일 최종혼잡도(인원수).csv')
friFin.to_csv('금요일 최종혼잡도(인원수).csv')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 1차혼잡도 정규화(PMF)
import io
import pandas as pd
import numpy as np


# 요일 별 1차혼잡도 데이터(행렬전환 불가)
from google.colab import drive
drive.mount('/content/drive')
monFin = pandas.read_csv('/content/drive/MyDrive/데이터톤/전처리/혼잡도 과정데이터(사용X)/월요일 최종혼잡도(인원수).csv')
monFin.drop(['Unnamed: 0'], axis = 1, inplace=True)
print(monFin)
tueFin = pandas.read_csv('/content/drive/MyDrive/데이터톤/전처리/혼잡도 과정데이터(사용X)/화요일 최종혼잡도(인원수).csv')
wedFin = pandas.read_csv('/content/drive/MyDrive/데이터톤/전처리/혼잡도 과정데이터(사용X)/수요일 최종혼잡도(인원수).csv')
thuFin = pandas.read_csv('/content/drive/MyDrive/데이터톤/전처리/혼잡도 과정데이터(사용X)/목요일 최종혼잡도(인원수).csv')
friFin = pandas.read_csv('/content/drive/MyDrive/데이터톤/전처리/혼잡도 과정데이터(사용X)/금요일 최종혼잡도(인원수).csv')

  

# 최소-최대 정규화
def minmax_norm(df_input):
    return df_input.apply(lambda x: (x-x.min())/ (x.max() - x.min()), axis=1)
monNorm_2 = minmax_norm(monFin)
print(monNorm_2)
tueNorm = minmax_norm(tueFin)
wedNorm = minmax_norm(wedFin)
thuNorm = minmax_norm(thuFin)
friNorm = minmax_norm(friFin)


# CSV 저장
monNorm_2.to_csv('월요일 최종혼잡도(정규화).csv')
tueNorm.to_csv('화요일 최종혼잡도(정규화).csv')
wedNorm.to_csv('수요일 최종혼잡도(정규화).csv')
thuNorm.to_csv('목요일 최종혼잡도(정규화).csv')
friNorm.to_csv('금요일 최종혼잡도(정규화).csv')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
           CJ법학관           L-P  ...         타이거플라자          환경실험관
0    3634.100407   2702.821775  ...    5489.667158    4337.541175
1    7268.200814   4922.643550  ...   10979.334315    8675.082350
2   10902.301221   7142.465325  ...   16469.001473   13012.623526
3   18764.252148  13668.793748  ...   30476.284732   24268.626833
4   23294.276119  16733.054258  ...   39194.590083   31982.449565
5   27824.300090  20280.314768  ...   47912.895434   39696.272296
6   35865.879466  26814.927776  ...   61949.804124   50959.527395
7   39441.535504  28561.219547  ...   67379.230731   55337.871018
8   43061.191542  31428.430111  ...   73067.440844   60559.952927
9   46645.394165  34269.206612  ...   78708.119293   65658.034836
10  54236.198100  41477.349982  ...   91607.419244   77063.874655
11  58076.799412  44809.150916  ...   98913.572410   83842.921288
12  61969.400

In [ ]:
# 2차 혼잡도 산정을 위한 외부 데이터 결합(캠퍼스 주변 대중교통 하차 인원을 고려한 가중치 부여)
import io
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
dfSub = pd.read_excel('/content/drive/MyDrive/데이터톤/data/서울교통공사 2019년 일별 역별 시간대별 승하차인원(1_8호선).xlsx', skiprows = [0]) # 지하철
dfBus = pd.read_csv('/content/drive/MyDrive/데이터톤/data/2019년 버스노선별 정류장별 시간대별 승하차 인원 정보.csv', encoding = "EUC-KR") # 버스



# 버스
dfBus1 = dfBus[(dfBus['역명'] == '고대앞') |  (dfBus['역명'] == '고려대역.고대앞삼거리') |  (dfBus['역명'] == '안암오거리') |  (dfBus['역명'] == '개운사입구') |
               (dfBus['역명'] == '고려대이공대.고대병원') | (dfBus['역명'] == '고려대이공대') | (dfBus['역명'] == '안암역') | (dfBus['역명'] == '고대병원') |
               (dfBus['역명'] == '안암전철역') | (dfBus['역명'] == '고려대안암학사') | (dfBus['역명'] == '고려대법과대학입구') | (dfBus['역명'] == '고려대아이스링크') |
               (dfBus['역명'] == '우신향병원') | (dfBus['역명'] == '고려대역3번출구')]
# 버스 하차
dfBus2 = dfBus1[['9시하차총승객수', '10시하차총승객수', '11시하차총승객수', '12시하차총승객수', '13시하차총승객수','14시하차총승객수',
                 '15시하차총승객수', '16시하차총승객수', '17시하차총승객수','18시하차총승객수', '19시하차총승객수', '20시하차총승객수',
                 '21시하차총승객수']]
dfBus2.rename(columns = {"9시하차총승객수": "09 ~ 10", "10시하차총승객수": "10 ~ 11", "11시하차총승객수": "11 ~ 12", "12시하차총승객수": "12 ~ 13", "13시하차총승객수": "13 ~ 14", 
                         "14시하차총승객수": "14 ~ 15", "15시하차총승객수": "15 ~ 16", "16시하차총승객수": "16 ~ 17", "17시하차총승객수": "17 ~ 18", "18시하차총승객수": "18 ~ 19",
                         "19시하차총승객수": "19 ~ 20", "20시하차총승객수": "20 ~ 21", "21시하차총승객수": "21 ~ 22"}, inplace = True)


# 버스 승차
dfBus25 = dfBus1[['9시승차총승객수', '10시승차총승객수', '11시승차총승객수', '12시승차총승객수', '13시승차총승객수','14시승차총승객수',
                 '15시승차총승객수', '16시승차총승객수', '17시승차총승객수','18시승차총승객수', '19시승차총승객수', '20시승차총승객수',
                 '21시승차총승객수']]
dfBus25.rename(columns = {"9시승차총승객수": "09 ~ 10", "10시승차총승객수": "10 ~ 11", "11시승차총승객수": "11 ~ 12", "12시승차총승객수": "12 ~ 13", "13시승차총승객수": "13 ~ 14", 
                         "14시승차총승객수": "14 ~ 15", "15시승차총승객수": "15 ~ 16", "16시승차총승객수": "16 ~ 17", "17시승차총승객수": "17 ~ 18", "18시승차총승객수": "18 ~ 19",
                         "19시승차총승객수": "19 ~ 20", "20시승차총승객수": "20 ~ 21", "21시승차총승객수": "21 ~ 22"}, inplace = True)

dfBusFin = dfBus2 - dfBus25
dfBusFin.loc['평균'] = dfBusFin.mean()
dfBusFinal = dfBusFin.tail(n=1) 
dfBusFinal.to_csv('버스평균.csv')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
# 외부 데이터 결합(캠퍼스 주변 대중교통 하차 인원을 고려한 가중치 부여)
import io
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
dfSub = pd.read_excel('/content/drive/MyDrive/데이터톤/data/서울교통공사 2019년 일별 역별 시간대별 승하차인원(1_8호선).xlsx', skiprows = [0]) # 지하철

# 지하철
num = [0, 1, 2, 3, 4]
for i in num:
  dfSub1 = dfSub[(dfSub['요일'] == i)] # 요일 결정
  dfSub2 = dfSub1[(dfSub1['역명'] == '안암(고대병원앞)') |  (dfSub1['역명'] == '고려대(종암)')]
  dfSub3 = dfSub2[(dfSub2['구분'] == '하차') &  (dfSub2['날짜'] < '2019-07-01') &  (dfSub2['날짜'] >= '2019-03-01')]
  dfSub4 = dfSub2[(dfSub2['구분'] == '승차') &  (dfSub2['날짜'] < '2019-07-01') &  (dfSub2['날짜'] >= '2019-03-01')]
  dfSub33 = dfSub3[['09 ~ 10', '10 ~ 11', '11 ~ 12', '12 ~ 13', '13 ~ 14', '14 ~ 15', '15 ~ 16', '16 ~ 17', '17 ~ 18', '18 ~ 19', '19 ~ 20','20 ~ 21', '21 ~ 22']]
  dfSub44 = dfSub4[['09 ~ 10', '10 ~ 11', '11 ~ 12', '12 ~ 13', '13 ~ 14', '14 ~ 15', '15 ~ 16', '16 ~ 17', '17 ~ 18', '18 ~ 19', '19 ~ 20','20 ~ 21', '21 ~ 22']]
  dfSub33.to_csv('지하철하차{}.csv'.format(i))
  dfSub44.to_csv('지하철승차{}.csv'.format(i))


df1 = pd.read_csv('/content/지하철하차4.csv') 
df2 = pd.read_csv('/content/지하철승차4.csv')
df0 = df1 - df2
df0.loc['평균'] = df0.mean()
df00 = df0.tail(n=1) 
df00.to_csv('지하철평균_금.csv') # CSV 저장

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import io
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
df0 = pd.read_csv('/content/drive/MyDrive/데이터톤/전처리/혼잡도 과정데이터(사용X)/지하철평균_월.csv') 
df1 = pd.read_csv('/content/drive/MyDrive/데이터톤/전처리/혼잡도 과정데이터(사용X)/지하철평균_화.csv') 
df2 = pd.read_csv('/content/drive/MyDrive/데이터톤/전처리/혼잡도 과정데이터(사용X)/지하철평균_수.csv') 
df3 = pd.read_csv('/content/drive/MyDrive/데이터톤/전처리/혼잡도 과정데이터(사용X)/지하철평균_목.csv') 
df4 = pd.read_csv('/content/drive/MyDrive/데이터톤/전처리/혼잡도 과정데이터(사용X)/지하철평균_금.csv') 
df5 = pd.read_csv('/content/drive/MyDrive/데이터톤/전처리/혼잡도 과정데이터(사용X)/버스평균.csv') 

df0 = df0.drop(columns=["Unnamed: 0", "Unnamed: 0.1"])
df1 = df1.drop(columns=["Unnamed: 0", "Unnamed: 0.1"])
df2 = df2.drop(columns=["Unnamed: 0", "Unnamed: 0.1"])
df3 = df3.drop(columns=["Unnamed: 0", "Unnamed: 0.1"])
df4 = df4.drop(columns=["Unnamed: 0", "Unnamed: 0.1"])
df5 = df5.drop(columns=["Unnamed: 0"])

dfMon = df0 + df5
dfTue = df1 + df5
dfWed = df2 + df5
dfThu = df3 + df5
dfFri = df4 + df5



# 최소-최대 정규화
def minmax_norm(df_input):
    return df_input.apply(lambda x: (x-x.min())/ (x.max() - x.min()), axis=1)
dfM = (minmax_norm(dfMon)).transpose()
dfTu = (minmax_norm(dfTue)).transpose()
dfW = (minmax_norm(dfWed)).transpose()
dfTh = (minmax_norm(dfThu)).transpose()
dfF = (minmax_norm(dfFri)).transpose()
print(dfM)
print(dfTu)
print(dfW)
print(dfTh)
print(dfF)




# 수강인원으로 산출한 최종혼잡도
df6 = pd.read_csv('/content/drive/MyDrive/데이터톤/전처리/혼잡도 과정데이터(사용X)/월요일 최종혼잡도(정규화).csv')
df7 = pd.read_csv('/content/drive/MyDrive/데이터톤/전처리/혼잡도 과정데이터(사용X)/화요일 최종혼잡도(정규화).csv')
df8 = pd.read_csv('/content/drive/MyDrive/데이터톤/전처리/혼잡도 과정데이터(사용X)/수요일 최종혼잡도(정규화).csv')
df9 = pd.read_csv('/content/drive/MyDrive/데이터톤/전처리/혼잡도 과정데이터(사용X)/목요일 최종혼잡도(정규화).csv')
df10 = pd.read_csv('/content/drive/MyDrive/데이터톤/전처리/혼잡도 과정데이터(사용X)/금요일 최종혼잡도(정규화).csv')
df11 = pd.read_csv('/content/drive/MyDrive/데이터톤/전처리/혼잡도 과정데이터(사용X)/월_대중교통총합.CSV', encoding = "EUC-KR") # 대중교통 혼잡도 전처리 파일1
df12 = pd.read_csv('/content/drive/MyDrive/데이터톤/전처리/혼잡도 과정데이터(사용X)/화_대중교통총합.CSV', encoding = "EUC-KR") # 대중교통 혼잡도 전처리 파일2
df13 = pd.read_csv('/content/drive/MyDrive/데이터톤/전처리/혼잡도 과정데이터(사용X)/수_대중교통총합.CSV', encoding = "EUC-KR") # 대중교통 혼잡도 전처리 파일2
df14 = pd.read_csv('/content/drive/MyDrive/데이터톤/전처리/혼잡도 과정데이터(사용X)/목_대중교통총합.CSV', encoding = "EUC-KR") # 대중교통 혼잡도 전처리 파일2
df15 = pd.read_csv('/content/drive/MyDrive/데이터톤/전처리/혼잡도 과정데이터(사용X)/금_대중교통총합.CSV', encoding = "EUC-KR") # 대중교통 혼잡도 전처리 파일2


# 가중치 더하기
Mon = df6 + df11
Tue = df7 + df12
Wed = df8 + df13
Thu = df9 + df14
Fri = df10 + df15


Mon = Mon.drop(columns=["Unnamed: 0"])
Tue = Tue.drop(columns=["Unnamed: 0", "Unnamed: 0.1"])
Wed = Wed.drop(columns=["Unnamed: 0", "Unnamed: 0.1"])
Thu = Thu.drop(columns=["Unnamed: 0", "Unnamed: 0.1"])
Fri = Fri.drop(columns=["Unnamed: 0", "Unnamed: 0.1"])

MonF = minmax_norm(Mon)
TueF = minmax_norm(Tue)
WedF = minmax_norm(Wed)
ThuF = minmax_norm(Thu)
FriF = minmax_norm(Fri)


# 최종 혼잡도
MonF.to_csv('혼잡도_월.csv')
TueF.to_csv('혼잡도_화.csv')
WedF.to_csv('혼잡도_수.csv')
ThuF.to_csv('혼잡도_목.csv')
FriF.to_csv('혼잡도_금.csv')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
                0
09 ~ 10  0.841961
10 ~ 11  1.000000
11 ~ 12  0.601359
12 ~ 13  0.459118
13 ~ 14  0.557050
14 ~ 15  0.387439
15 ~ 16  0.233476
16 ~ 17  0.066709
17 ~ 18  0.000000
18 ~ 19  0.160556
19 ~ 20  0.366769
20 ~ 21  0.262669
21 ~ 22  0.169996
                0
09 ~ 10  0.797618
10 ~ 11  1.000000
11 ~ 12  0.579656
12 ~ 13  0.415293
13 ~ 14  0.521888
14 ~ 15  0.366156
15 ~ 16  0.229301
16 ~ 17  0.053316
17 ~ 18  0.000000
18 ~ 19  0.130924
19 ~ 20  0.350007
20 ~ 21  0.244319
21 ~ 22  0.158215
                0
09 ~ 10  0.810694
10 ~ 11  1.000000
11 ~ 12  0.590082
12 ~ 13  0.441858
13 ~ 14  0.522723
14 ~ 15  0.379533
15 ~ 16  0.180548
16 ~ 17  0.051337
17 ~ 18  0.000000
18 ~ 19  0.174657
19 ~ 20  0.364192
20 ~ 21  0.256981
21 ~ 22  0.181718
                0
09 ~ 10  0.791085
10 ~ 11  1.000000
11 ~ 12  0.583042
12 ~ 13  0.433705
13 ~ 14  0.539427
14 ~ 15